In [6]:
import os
import shutil
import random
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import splitfolders
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import tensorflow as tf

In [7]:
# Base data directory
base_dir = Path('/Users/tehreem/Desktop/Study/Projects/defungi')
source_dir = base_dir / 'data'

In [8]:
# Calculating class weights before augmentation
# y_train is the list/array of original training labels (before augmentation)
train_dir = Path(base_dir/'output/train')  # path to your training data folder

# Step 1: Collect labels
y_train = []
for class_dir in train_dir.iterdir():
    if class_dir.is_dir():
        label = class_dir.name
        for img_file in class_dir.iterdir():
            if img_file.is_file() and img_file.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                y_train.append(label)

# Step 2: Calculate class weights
classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))

print(class_weight_dict)

{'H1': 0.4138518308260006, 'H2': 0.7809319764327799, 'H3': 2.2259541984732825, 'H5': 2.229357798165138, 'H6': 2.467005076142132}


In [9]:
IMG_SIZE = 224
BATCH_SIZE = 32

train_dir = Path(base_dir/'output/train')
val_dir = Path(base_dir/'output/val')
test_dir = Path(base_dir/'output/test')

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

class_names = train_ds.class_names

Found 7290 files belonging to 5 classes.
Found 908 files belonging to 5 classes.
Found 916 files belonging to 5 classes.


In [10]:
IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS = 224, 224, 3
BATCH_SIZE = 32

# 1. Light augmentation pipeline for majority classes (without rescaling here)
light_aug = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    # no Rescaling here; will do once after augmentation
])

# 2. Aggressive augmentation pipeline for minority classes
# If TF < 2.6, split horizontal_and_vertical into two layers
aggressive_aug = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomFlip("vertical"),
    tf.keras.layers.RandomRotation(0.3),
    tf.keras.layers.RandomZoom(0.4),
    tf.keras.layers.RandomContrast(0.2),
    # no Rescaling here; will do once after augmentation
])

minority_classes = ['H3', 'H5', 'H6']
minority_indices = [class_names.index(cls) for cls in minority_classes]
minority_indices_tensor = tf.constant(minority_indices, dtype=tf.int64)

def class_specific_aug(image, label):
    class_idx = tf.argmax(label)
    is_minority = tf.reduce_any(tf.equal(class_idx, minority_indices_tensor))

    def aggressive():
        img = aggressive_aug(image)
        img.set_shape([IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS])
        return img, label

    def light():
        img = light_aug(image)
        img.set_shape([IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS])
        return img, label

    return tf.cond(is_minority, aggressive, light)

# Load datasets (from your original code)
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

# Apply augmentation by unbatching, augmenting, rescaling once, batching, and prefetching
aug_train_ds = (
    train_ds
    .unbatch()
    .map(class_specific_aug, num_parallel_calls=tf.data.AUTOTUNE)
    # Rescale after augmentation, to avoid double scaling
    .map(lambda x, y: (tf.image.convert_image_dtype(x, tf.float32), y), num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(1000)  # Shuffle after augmentation
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

# Define repeat factors for classes
repeat_factors = {
    'H1': 1,
    'H2': 2,
    'H3': 5,
    'H5': 5,
    'H6': 6
}

datasets = []
for i, class_name in enumerate(class_names):
    class_ds = (
        aug_train_ds
        .unbatch()
        .filter(lambda x, y: tf.equal(tf.argmax(y), i))
        .shuffle(1000)  # Shuffle per-class dataset
        .batch(BATCH_SIZE)
        .repeat(repeat_factors[class_name])
    )
    datasets.append(class_ds)

balanced_train_ds = tf.data.experimental.sample_from_datasets(datasets)
balanced_train_ds = balanced_train_ds.prefetch(tf.data.AUTOTUNE)

# Optional: enable eager mode globally during debugging to avoid crashes
# tf.config.run_functions_eagerly(True)


Found 7290 files belonging to 5 classes.
Instructions for updating:
Use `tf.data.Dataset.sample_from_datasets(...)`.


In [11]:
for batch in balanced_train_ds.take(1):
    # Usually, batch is a tuple: (images, labels)
    images, labels = batch
    
    print(f"Images type: {type(images)}")
    print(f"Images shape: {images.shape}")
    
    print(f"Labels type: {type(labels)}")
    print(f"Labels shape: {labels.shape}")
    
    # For labels, to see values:
    print("Labels sample (numpy):", labels.numpy()[:5])


2025-06-09 13:03:01.281171: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


Images type: <class 'tensorflow.python.framework.ops.EagerTensor'>
Images shape: (32, 224, 224, 3)
Labels type: <class 'tensorflow.python.framework.ops.EagerTensor'>
Labels shape: (32, 5)
Labels sample (numpy): [[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]]


2025-06-09 13:03:05.900522: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:
for sample in balanced_train_ds.take(1):
    print("Data types:", type(sample), " | Length:", len(sample))
    print("Data types inside batch:", [type(i) for i in sample])

#So these lines are not able to execute
images, labels = next(iter(balanced_train_ds))
print("Label dtype:", labels.dtype)
print("Label sample:", labels[:5])

Data types: <class 'tuple'>  | Length: 2
Data types inside batch: [<class 'tensorflow.python.framework.ops.EagerTensor'>, <class 'tensorflow.python.framework.ops.EagerTensor'>]


2025-06-09 13:04:39.991835: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Label dtype: <dtype: 'float32'>
Label sample: tf.Tensor(
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]], shape=(5, 5), dtype=float32)


2025-06-09 13:04:42.123266: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{function_node cond_true_279}}{{node cond/sequential_5_1/random_flip_7_1/Add}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2025-06-09 13:04:42.123283: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{function_node cond_true_279}}{{node cond/sequential_5_1/random_flip_7_1/Add}}]]
	 [[cond/then/_0/cond/sequential_5_1/random_contrast_2_1/Add/ReadVariableOp/_32]] [type.googleapis.com/tensorflow.DerivedStatus='']
2025-06-09 13:04:42.123290: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11020564164073132315
2025-06-09 13:04:42.123293: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 10224367694141399667
2025-06-09 13:04:42.123295: I 

In [13]:
for images, labels in balanced_train_ds.take(1):
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)
    print("Sample labels:", labels.numpy()[:5])

Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 5)
Sample labels: [[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]


2025-06-09 13:05:08.620956: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{function_node cond_true_279}}{{node cond/sequential_5_1/random_flip_8_1/Add}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2025-06-09 13:05:08.620989: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 189364043324911740
2025-06-09 13:05:08.621001: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 1395017209629196588
2025-06-09 13:05:08.621008: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{function_node cond_true_279}}{{node cond/sequential_5_1/random_flip_8_1/Add}}]]
	 [[cond/then/_0/cond/sequential_5_1/random_flip_7_1/ReadVariableOp/_22]] [type.googleapis.com/tensorflow.DerivedStatus='']
2025-06-09 13:05:08.621014: I tensorflow/

In [14]:
# Prepare Validation and Test Sets do not use augmentation—just rescale:
rescale = tf.keras.Sequential([tf.keras.layers.Rescaling(1./255)])

val_ds = val_ds.map(lambda x, y: (rescale(x), y)).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(lambda x, y: (rescale(x), y)).prefetch(tf.data.AUTOTUNE)

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [17]:
# original dict with string keys
class_weight_dict_str = {
    'H1': 0.4138518308260006,
    'H2': 0.7809319764327799,
    'H3': 2.2259541984732825,
    'H5': 2.229357798165138,
    'H6': 2.467005076142132
}

# Convert to integer keys for Keras
class_weight_dict = {
    class_names.index(cls): float(weight)
    for cls, weight in class_weight_dict_str.items()
}

print(class_weight_dict)

{0: 0.4138518308260006, 1: 0.7809319764327799, 2: 2.2259541984732825, 3: 2.229357798165138, 4: 2.467005076142132}


In [18]:
model.fit(
    balanced_train_ds,
    validation_data=val_ds,
    epochs=10,
    class_weight=class_weight_dict  # Use the converted class weights
)

Epoch 1/10
    552/Unknown 120s 205ms/step - accuracy: 0.2650 - loss: 2357.8589

2025-06-09 13:08:13.493100: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-06-09 13:08:13.493120: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_14]]
2025-06-09 13:08:13.493132: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 5326962178744273090
2025-06-09 13:08:13.493141: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9218567080647906154
2025-06-09 13:08:13.493144: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 4464310331644997555
2025-06-09 13:08:13.493150: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 15533628806704469064
2025-06-0

553/553 ━━━━━━━━━━━━━━━━━━━━ 120s 206ms/step - accuracy: 0.2651 - loss: 2351.4929 - val_accuracy: 0.0969 - val_loss: 1.6498
Epoch 2/10
552/553 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 0.2922 - loss: 2.3477

2025-06-09 13:10:20.506462: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-06-09 13:10:20.506493: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_42]]
2025-06-09 13:10:20.506497: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 4081508238768780211
2025-06-09 13:10:20.506499: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 6744818974853584469
2025-06-09 13:10:20.506505: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 645114054710323013
2025-06-09 13:10:20.506507: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 18157025256602548069
2025-06-09

553/553 ━━━━━━━━━━━━━━━━━━━━ 127s 220ms/step - accuracy: 0.2922 - loss: 2.3480 - val_accuracy: 0.0925 - val_loss: 1.7115
Epoch 3/10
552/553 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step - accuracy: 0.2703 - loss: 2.2382

...

201/553 ━━━━━━━━━━━━━━━━━━━━ 1:59 339ms/step - accuracy: 0.2861 - loss: 2.0754

2025-06-09 13:30:30.346633: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


552/553 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step - accuracy: 0.2959 - loss: 2.0789

2025-06-09 13:32:01.171642: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-06-09 13:32:01.171673: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/adam/Add_8/ReadVariableOp/_1]]
2025-06-09 13:32:01.171685: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16285674403355951032
2025-06-09 13:32:01.171699: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 4081508238768780211
2025-06-09 13:32:01.171707: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 643988392482176096
2025-06-09 13:32:01.171713: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key has

553/553 ━━━━━━━━━━━━━━━━━━━━ 167s 288ms/step - accuracy: 0.2960 - loss: 2.0790 - val_accuracy: 0.0892 - val_loss: 1.9962


In [19]:
#Evaluate Model on Validation Set
val_loss, val_acc = model.evaluate(val_ds)
print(f"Validation accuracy: {val_acc:.2f}")

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0833 - loss: 2.0060
Validation accuracy: 0.09
